## Rewrite of the "Baselines for mortality and LOS prediction" notebooks.
#### This notebook contains code to predict LOS using GRU-D, Random Forest and Logistic Regression in a simple notebook with explanations.

Author(s): Tomass Wilson, thwmi@kth.se

Credit: Shirly Wang, Matthew B. A. McDermott, Geeticka Chauhan, Michael C. Hughes, Tristan Naumann, 
and Marzyeh Ghassemi. MIMIC-Extract: A Data Extraction, Preprocessing, and Representation 
Pipeline for MIMIC-III. arXiv:1907.08322. 

### Setup

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload

import copy, math, os, pickle, time, pandas as pd, numpy as np, scipy.stats as ss
import warnings
import datetime
from IPython.display import clear_output

from matplotlib import pyplot as plt
from matplotlib import ticker
from matplotlib import font_manager as fm
import matplotlib

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score, auc, roc_curve

import torch, torch.utils.data as utils, torch.nn as nn, torch.nn.functional as F, torch.optim as optim
from torch.autograd import Variable
from torch.nn.parameter import Parameter


from mmd_grud_utils import *

In [ ]:
dirname = os.getcwd()
DATA_FILEPATH     = os.path.join(dirname, "..", "data", "all_hourly_data.h5")

In [ ]:
GAP_TIME          = 6  # In hours
WINDOW_SIZE       = 24 # In hours
SEED              = 2
ID_COLS           = ['subject_id', 'hadm_id', 'icustay_id']
GPU               = '0'  # set this to the ID(s) of your most powerful GPU(s)

os.environ['CUDA_VISIBLE_DEVICES'] = GPU
np.random.seed(SEED)
torch.manual_seed(SEED)

### Load Data

In [ ]:
%%time
data_full = pd.read_hdf(DATA_FILEPATH, 'vitals_labs')
statics        = pd.read_hdf(DATA_FILEPATH, 'patients')

In [ ]:
data_full.head()

### Create True/False mappings of the targets LOS 3/7 days
Also we create a list of subjects that all have lengths of stay that are at least 30 hours long.

Ys is a dataframe containing the target results for each subject (eg that they did stay for longer than 3 days).

data_df contains the metrics which we want to evaluate to predict length of stay. They are also the ones on which differential privacy will be applied.

In [ ]:
# All patients in ys ahave been in icu at some point for at least 30 hours
Ys = statics[statics.max_hours > WINDOW_SIZE + GAP_TIME][['los_icu']]
Ys['los_3'] = Ys['los_icu'] > 3  # True/False column
Ys['los_7'] = Ys['los_icu'] > 7

# Get only the medical data of the chosen subjects
data_df = data_full[
    (data_full.index.get_level_values('icustay_id').isin(set(Ys.index.get_level_values('icustay_id')))) &
    (data_full.index.get_level_values('hours_in') < WINDOW_SIZE)
]

# Collect the subject id's
subj_idx, Ys_subj_idx = [df.index.get_level_values('subject_id') for df in (data_df, Ys)]
ids = set(subj_idx)
assert ids == set(Ys_subj_idx), "Subject ID pools differ!"

In [ ]:
Ys.sum()

In [ ]:
Ys

### Squash data
Squash the data into 1 measurement per person. Also gets rid of the infuriating column multiindex

In [ ]:
def squash(df):
    sq = df.swaplevel(0, 1, axis=1).stack()
    squashed = sq.groupby(ID_COLS + ["LEVEL2"]).agg({"mean": "mean"}).unstack()
    squashed = squashed.fillna(0)  # Not the best, but ehhhhhh
    
    return squashed

### Add differential privacy
We will use variable privacy budget to see how changes affect performance

We add noise to train and dev sets, as they are used in training, but not to test so as to get accurate results. IRL, test would also not be able to have local diff privacy (just as with actual used data)

The noise is added to all values, NaNs stay as NaNs (Noisy values will be propagated to NaNs later)

In [ ]:
def sensitivity(feature):
    """
    Find the sensitivity
    This version groups people by subject ID and sums their measured absolute values (L1Norm)
    It then takes the max across all subjects to give the sensitivity. It treats NaN values as 0.
    
    returns:
        max_l1: a float representing this function's sensitivity
    """
    
    return np.nanmax(feature.abs())

In [ ]:
def add_noise(df, epsilon):
    """
    Add noise to the mimic 3 dataset
    
    Args:
        df: The dataframe on which to add noise, as formatted  by mimic extract
        epsilon: The differential privacy budget to use
        
    returns:
        noisy_df: The dataframe with added noise
    """
    
    generator = np.random.default_rng()
    noisy_df = df.copy()
    idx = pd.IndexSlice
    
    for feature_name in df["mean"]:
        feature =  df.loc[:, ("mean", feature_name)].copy()
        
        with warnings.catch_warnings():
            warnings.filterwarnings('error')
            sens = sensitivity(feature)
            
        scale = sens / epsilon  # Definition of scale parameter for laplace noise to fulfill diff privacy
        noise = generator.laplace(0, scale, len(feature))
        
        noisy_feature = feature + noise
        noisy_df.loc[:, ("mean", feature_name)] = noisy_feature
        
    return noisy_df

def add_ys_noise(Ys, epsilon):
    """
    Add noise to the mimic 3 dataset Ys
    
    Args:
        Ys: The dataframe on which to add noise, wit an los_icu column
        epsilon: The differential privacy budget to use
        
    returns:
        noisy_df: The dataframe with added noise
    """
    
    generator = np.random.default_rng()
    noisy_Ys = Ys.copy()
    
    feature = Ys.loc[:, "los_icu"].copy()
    with warnings.catch_warnings():
        warnings.filterwarnings('error')
        sens = sensitivity(feature)

    scale = sens / epsilon  # Definition of scale parameter for laplace noise to fulfill diff privacy
    noise = generator.laplace(0, scale, len(feature))

    noisy_feature = feature + noise
    noisy_Ys.loc[:, "los_icu"] = noisy_feature
    
    Ys['los_3'] = Ys['los_icu'] > 3  # True/False column
    Ys['los_7'] = Ys['los_icu'] > 7
        
    return noisy_Ys

### Create train, dev and test fractions of the data

In [ ]:
def create_test_sets(data_df, Ys):
    test_frac = 0.2
    np.random.seed(SEED)
    subjects, N = np.random.permutation(list(ids)), len(ids)
    N_test = int(test_frac * N)
    test_subj = subjects[:N_test]
    remainder_subj = subjects[N_test:]

    # Create train, dev and test fractions for data and Ys
    [(data_remainder, data_test), (Ys_remainder, Ys_test)] = [
        [df[df.index.get_level_values('subject_id').isin(s)].copy() for s in (remainder_subj, test_subj)] \
        for df in (data_df, Ys)
    ]
    return data_remainder, Ys_remainder, data_test, Ys_test

In [ ]:
def create_train_dev_sets(data_df, Ys):
    train_frac, dev_frac = 0.875, 0.125,
    np.random.seed(SEED)
    subjects, N = np.random.permutation(list(ids)), len(ids)
    N_train, N_dev = int(train_frac * N), int(dev_frac * N)
    train_subj = subjects[:N_train]
    dev_subj  = subjects[N_train:]

    # Create train, dev and test fractions for data and Ys
    [(data_train, data_dev), (Ys_train, Ys_dev)] = [
        [df[df.index.get_level_values('subject_id').isin(s)].copy() for s in (train_subj, dev_subj)] \
        for df in (data_df, Ys)
    ]
    return data_train, data_dev, Ys_train, Ys_dev

### Normalization
Normalise all features/measurements to be the number of standard deviations from the mean

In [ ]:
def normalize_datasets(data_train, data_dev, data_test):
    idx = pd.IndexSlice
    data_means = data_train.loc[:, idx['mean', :]].mean(axis=0)
    data_stds = data_train.loc[:, idx['mean', :]].std(axis=0)

    data_train.loc[:, idx['mean', :]] = (data_train.loc[:, idx['mean', :]] - data_means)/data_stds
    data_dev.loc[:, idx['mean', :]] = (data_dev.loc[:, idx['mean', :]] - data_means)/data_stds
    data_test.loc[:, idx['mean', :]] = (data_test.loc[:, idx['mean', :]] - data_means)/data_stds
    
def norm_data(data_df):
    idx = pd.IndexSlice
    normed = data_df.copy()
    data_means = normed.loc[:, idx['mean', :]].mean(axis=0)
    data_stds = normed.loc[:, idx['mean', :]].std(axis=0)

    normed.loc[:, idx['mean', :]] = (normed.loc[:, idx['mean', :]] - data_means)/data_stds
    return normed

### Propagate most recent measurement to NaN values
Here the most recent value is propagated forward to cover and remove all the NaN values

In [ ]:
def simple_imputer(df):
    idx = pd.IndexSlice
    if len(df.columns.names) > 2: 
        df.columns = df.columns.droplevel(('label', 'LEVEL1', 'LEVEL2'))
    
    df_out = df.loc[:, idx[:, ['mean', 'count']]].copy()
    icustay_means = df_out.loc[:, 'mean'].groupby(ID_COLS).mean()
    
    df_out.loc[:,'mean'] = df_out.loc[:,'mean'].groupby(ID_COLS).fillna(
        method='ffill'
    ).groupby(ID_COLS).fillna(icustay_means).fillna(0)
    
    df_out.loc[:, idx[:, 'count']] = (df.loc[:, idx[:, 'count']] > 0).astype(float)
    df_out.rename(columns={'count': 'mask'}, level='Aggregation Function', inplace=True)
    
    is_absent = (1 - df_out.loc[:, idx[:, 'mask']])
    hours_of_absence = is_absent.cumsum()
    time_since_measured = hours_of_absence - hours_of_absence[is_absent==0].fillna(method='ffill')
    time_since_measured.rename(columns={'mask': 'time_since_measured'}, level='Aggregation Function', inplace=True)

    df_out = pd.concat((df_out, time_since_measured), axis=1)
    df_out.loc[:, idx[:, 'time_since_measured']] = df_out.loc[:, idx[:, 'time_since_measured']].fillna(100)
    
    df_out.sort_index(axis=1, inplace=True)
    return df_out

#data_train, data_dev, data_test = [
#    simple_imputer(df) for df in (data_train, data_dev, data_test)
#]

### Flatten Arrays
This is necessary because the models want their trainable features as columns. To train on the timeseries we flip "hours_in" onto the columns (instead of it being its own column)

This is *not* necessary for squashed data

In [ ]:
def flatten_arrays(data_train, data_dev, data_test):
    data_flat_train, data_flat_dev, data_flat_test = [
        df.pivot_table(index=ID_COLS, columns=['hours_in']) for df in (
            data_train, data_dev, data_test
        )
    ]
    return data_flat_train, data_flat_dev, data_flat_test

## Prediction

### Hyperparameters

In [ ]:
class DictDist():
    def __init__(self, dict_of_rvs): self.dict_of_rvs = dict_of_rvs
    def rvs(self, n):
        a = {k: v.rvs(n) for k, v in self.dict_of_rvs.items()}
        out = []
        for i in range(n): out.append({k: vs[i] for k, vs in a.items()})
        return out

class Choice():
    def __init__(self, options): self.options = options
    def rvs(self, n): return [self.options[i] for i in ss.randint(0, len(self.options)).rvs(n)]

In [ ]:
N = 15

LR_dist = DictDist({
    'C': Choice(np.geomspace(1e-3, 1e3, 10000)),
    'penalty': Choice(['l1', 'l2']),
    'solver': Choice(['liblinear', 'lbfgs']),
    'max_iter': Choice([100, 500])
})
np.random.seed(SEED)
LR_hyperparams_list = LR_dist.rvs(N)
for i in range(N):
    if LR_hyperparams_list[i]['solver'] == 'lbfgs': LR_hyperparams_list[i]['penalty'] = 'l2'

RF_dist = DictDist({
    'n_estimators': ss.randint(50, 500),
    'max_depth': ss.randint(2, 10),
    'min_samples_split': ss.randint(2, 75),
    'min_samples_leaf': ss.randint(1, 50),
})
np.random.seed(SEED)
RF_hyperparams_list = RF_dist.rvs(N)

GRU_D_dist = DictDist({
    'cell_size': ss.randint(50, 75),
    'hidden_size': ss.randint(65, 95),
    'learning_rate': ss.uniform(2e-3, 1e-1),
    'num_epochs': ss.randint(60, 150),
    'patience': ss.randint(5, 9),
    'batch_size': Choice([128, 256, 512]),
    'early_stop_frac': ss.uniform(0.05, 0.1),
    'seed': ss.randint(1, 10000),
})
np.random.seed(SEED)
GRU_D_hyperparams_list = GRU_D_dist.rvs(N)

### Cumulative accuracy profile (CAP) &  Accuracy ratio (AR)
Nice metric for low prevalence data. CAP is a graph, with a list of samples sorted by their predicted probabilities on the x axis and cumulative true positive rate on the y axis.
AR is the ratio of area under a perfect classifier to the area under the model.



In [ ]:
def CAP(y_true, y_score):
    total = len(y_true)
    
    generator = np.random.default_rng()
    y_random = generator.random(total)
    
    def take_first_random(item):
        return (item[0], item[2])  # the third item is a random variable, to jumble the answers
    
    model_y = [y for _, y, _ in sorted(zip(y_score, y_true, y_random), reverse = True, key=take_first_random)]
    y_values = np.append([0], np.cumsum(model_y))
    x_values = np.arange(0, total + 1)
    return x_values, y_values
    
def AR(y_true, y_score):
    total = len(y_true)
    prevalence = sum(y_true)
    
    # Area under Random Model
    a = auc([0, total], [0, prevalence])

    # Area between Perfect and Random Model
    aP = auc([0, prevalence, total], [0, prevalence, prevalence]) - a

    # Area between Trained and Random Model
    x_values, y_values = CAP(y_true, y_score)
    aT = auc(x_values, y_values)
    aR = aT - a
    
    return (aR / aP)

### Random Forest and Logistic Regression

In [ ]:
def update_progress(progress, update, replace=0):
    if replace > 0: 
        for _ in range(replace): progress.pop()
        progress.append(update)
        clear_output(wait=True)
        print(*progress, sep="\n")
    else:
        progress.append(update)
        print(update)

def run_basic(model, hyperparams_list, X_flat_train, X_flat_dev, X_flat_test, target, Ys_train, Ys_dev, Ys_test, progress):
    best_s, best_hyperparams = -np.Inf, None
    for i, hyperparams in enumerate(hyperparams_list):
        update_progress(progress, "On sample %d / %d (hyperparams = %s)" % (i+1, len(hyperparams_list), repr((hyperparams))), replace=1 if i>0 else 0)
        M = model(**hyperparams)
        M.fit(X_flat_train, Ys_train[target])
        s = roc_auc_score(Ys_dev[target], M.predict_proba(X_flat_dev)[:, 1])
        if s > best_s:
            best_s, best_hyperparams = s, hyperparams
            #print("New Best Score: %.2f @ hyperparams = %s" % (100*best_s, repr((best_hyperparams))))

    return run_only_final(model, best_hyperparams, X_flat_train, X_flat_dev, X_flat_test, target, Ys_train, Ys_dev, Ys_test)

def run_only_final(model, best_hyperparams, X_flat_train, X_flat_dev, X_flat_test, target, Ys_train, Ys_dev, Ys_test):
    best_M = model(**best_hyperparams)
    best_M.fit(pd.concat((X_flat_train, X_flat_dev)), pd.concat((Ys_train, Ys_dev))[target])
    y_true  = Ys_test[target]
    
    # calculate roc curves
    y_true_dev = Ys_dev[target]
    y_score_dev = best_M.predict_proba(X_flat_dev)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_true_dev, y_score_dev)
    # get the best threshold
    J = tpr - fpr
    ix = np.argmax(J)
    best_thresh = thresholds[ix]  # This best threshold should help to maximise accuracy and F1
    
    y_score = best_M.predict_proba(X_flat_test)[:, 1]
    y_pred  = (y_score >= best_thresh).astype('int')
    y_pred = best_M.predict(X_flat_test)
   
    result = {
        #"model":  best_M,
        "Params":   best_hyperparams,
        "AUROC":    roc_auc_score(y_true, y_score),
        "AUPRC":    average_precision_score(y_true, y_score),
        "Accuracy": accuracy_score(y_true, y_pred),
        "F1":       f1_score(y_true, y_pred),
        "AR":       AR(y_true, y_score),
    }
    
    return result

def save_result(task, model, epsilon, result, results, path):
    # Add result to results DataFrame
    result.update({"Task": task, "Model": model, "Epsilon": epsilon})
    results = results.append(result, ignore_index=True)
    
    # Sort by custom order
    results["Epsilon order"] = results["Epsilon"].apply(lambda x: "Z" if x == "Benchmark" else x) # makes sure Benchmark comes before Control
    results = results.sort_values(by=["Task", "Model", "Epsilon order"], ascending=[True, False, False])
    results = results.drop("Epsilon order", axis=1)
    results.to_csv(path, index=False)
    return results

def load_results(path):
    try:
        results = pd.read_csv(path, dtype={"Epsilon": object})
    except FileNotFoundError:
        BENCHMARK_PATH = os.path.join(dirname, "..", "data", "benchmark.csv")
        results = pd.read_csv(BENCHMARK_PATH, dtype={"Epsilon": object})
        results.to_csv(path, index=False)
    return results

### Keras NN model

In [ ]:
%%time
# https://machinelearningmastery.com/binary-classification-tutorial-with-the-keras-deep-learning-library/
os.environ["TF_GPU_THREAD_MODE"]="gpu_private"
os.environ["TF_XLA_FLAGS"]="--tf_xla_auto_jit=2"

import tensorflow as tf
options = tf.data.Options()
options.experimental_optimization.apply_default_optimizations = True
options.experimental_optimization.autotune_buffers = True
options.experimental_optimization.autotune = True
options.experimental_optimization.autotune_cpu_budget = 6
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
%load_ext tensorboard
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_validate
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# baseline model
def create_baseline(optimizer='RMSprop', hidden_size=100):
	# create model
    model = Sequential()
    model.add(Dense(104, input_dim=104, activation='relu'))
    model.add(Dense(hidden_size, activation='relu'))
    model.add(Dense(hidden_size, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
	# Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[tf.keras.metrics.AUC(curve='ROC', name="AUROC")], steps_per_execution=1)
    return model

def MLP(data, data_test, Ys, Ys_test, target, progress):
    
    encoder = LabelEncoder()
    encoder.fit(Ys[target])
    encoded_Y = encoder.transform(Ys[target])
    estimator = KerasClassifier(build_fn=create_baseline, epochs=30, batch_size=32, verbose=0)
    kfold = StratifiedKFold(n_splits=5, shuffle=True)
    scoring = ["roc_auc", "average_precision", "accuracy", "f1"]
    
    param_grid = dict(epochs=[10,20,30,40,50], batch_size=[16, 32, 64], optimizer=['RMSprop'], hidden_size=[20, 50, 100, 200, 400, 600])
    scv = RandomizedSearchCV(estimator=estimator, param_distributions=param_grid, n_iter=15, scoring=scoring, n_jobs=1, cv=kfold, refit="roc_auc")
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        result = scv.fit(data, encoded_Y)
        best_estimator = scv.best_estimator_
        y_pred = best_estimator.predict(data_test, verbose=0)
    results = {
        #"model":  best_M,
        "Params":   result.best_params_,
        "AUROC":    roc_auc_score(Ys_test[target], y_pred),
        "AUPRC":    average_precision_score(Ys_test[target], y_pred),
        "Accuracy": accuracy_score(Ys_test[target], tf.round(y_pred)),
        "F1":       f1_score(Ys_test[target], tf.round(y_pred))
    }
    update_progress(progress, "best hyperparams = {})".format(str(result.best_params_)), replace=0)
    return results

### Generate epsilons infinitely within bounds (that are 10^n apart from eachother)

In [ ]:
def epsilon_generator(lower=0.001, upper=1000, max_epsilons=10000):
    num_done = 0
    done_epsilons = []
    
    # Do all orders of magnitude first
    for i in range(np.log10(lower).astype(int), np.log10(upper).astype(int)+1):
        eps = 10**i
        done_epsilons.append(eps)
        num_done += 1
        yield str(eps)
        
    # Continue with all halves (logarithmic) between done epsilons
    while num_done < max_epsilons:
        to_do_epsilons = done_epsilons
        done_epsilons = []
        for i, epsilon in enumerate(to_do_epsilons):
            done_epsilons.append(epsilon)
            if(i + 1 == len(to_do_epsilons)):
                break  # Stop at last epsilon
            
            next_ep = to_do_epsilons[i + 1]
            log_midway = 10**((np.log10(epsilon) + np.log10(next_ep)) / 2)
            
            done_epsilons.append(log_midway)
            num_done += 1
            yield str(log_midway)
        

### Prep data (Stuff that only needs to be done once even across epsilon)

In [ ]:
squashed_data = squash(data_df)

# Generate results CSV

In [ ]:
%autoreload
from mmd_grud_utils import *
# ------------------------------------ Choose Run parameters here -----------------------------------------------
OVERWRITE = False
RESULTS_PATH = os.path.join(dirname, "..", "data", "results.csv")
models = [
    ('LR', LogisticRegression, LR_hyperparams_list),
    ('RF', RandomForestClassifier, RF_hyperparams_list),  
]
tasks = ['los_3', 'los_7']
epsilons = epsilon_generator()
# --------------------------------------------------------------------------------------------------------------

# Set up initial results dataframe
results = load_results(RESULTS_PATH)
progress = []
if OVERWRITE:
    prev_results_indices = results.loc[(results['Task'].isin(tasks)) &
                                       (results['Model'].isin([m for m,_,_ in models])) & 
                                       (results['Epsilon'].isin(epsilons))].index
    results.drop(prev_results_indices, inplace=True) 

for epsilon in [1]:
    if not OVERWRITE:
        # Save time by not generating new datasets if we already have all results for this epsilon
        prev_results = results.loc[(results["Task"].isin(tasks)) & 
                                   (results["Model"].isin([m for m,_,_ in models])) &
                                   (results["Epsilon"] == epsilon)]
        if prev_results.shape[0] == (len(models)) * len(tasks):
            update_progress(progress, "Already have all results for epsilon %s\n" % epsilon)
            continue
    
    # Create differentially private equivalent dataset
    # TODO: add noise to Ys
    update_progress(progress, "Creating datasets with epsilon %s...\n" % epsilon)
    
    data_remainder, Ys_remainder, data_test, Ys_test = create_test_sets(squashed_data, Ys)
    
    private_df = add_noise(data_remainder, float(epsilon))
    private_Ys = add_ys_noise(Ys_remainder, float(epsilon))
                  
    # Feature engineering pipeline 
    data_train, data_dev, Ys_train, Ys_dev = create_train_dev_sets(private_df, private_Ys)
    normalize_datasets(data_train, data_dev, data_test)

    # These steps not necessary with squashing
    # data_train, data_dev, data_test = [simple_imputer(df) for df in (data_train, data_dev, data_test)]
    # data_flat_train, data_flat_dev, data_flat_test = flatten_arrays(data_train, data_dev, data_test)
    data_flat_train, data_flat_dev, data_flat_test = data_train, data_dev, data_test
    
    for df in data_train, data_dev, data_test: 
        assert not df.isnull().any().any()

    # Use only mean columns
    data_flat_train = data_flat_train.loc[:, pd.IndexSlice['mean', :]]
    data_flat_dev = data_flat_dev.loc[:, pd.IndexSlice['mean', :]]
    data_flat_test = data_flat_test.loc[:, pd.IndexSlice['mean', :]]
    
    for model_name, model, hyperparams_list in models:
        for task in tasks:
            if not OVERWRITE:
                # Skip already calculated results
                prev_result = results.loc[(results['Task'] == task) & (results['Model'] == model_name) & (results['Epsilon'] == epsilon)]
                if not prev_result.empty: 
                    update_progress(progress, "Already have results for target {} with model {} on epsilon {:.4f} \n".format(task, model_name, float(epsilon)))
                    continue

            # Run gridsearch
            update_progress(progress, "Running model {} on target {} with epsilon {:.4f} \n".format(model_name, task, float(epsilon)))
            result = run_basic(model, hyperparams_list, data_flat_train, data_flat_dev, data_flat_test, task, Ys_train, Ys_dev, Ys_test, progress)
            results = save_result(task, model_name, epsilon, result, results, RESULTS_PATH)
            
            update_progress(progress, '\033[1m' + "Final results for target {} with model {} on epsilon {:.4f} \n".format(task, model_name, float(epsilon)) + '\033[0m', replace=2)
            update_progress(progress, result.pop("Params"))
            update_progress(progress, result)
            update_progress(progress, "")
            
        

##### add_noise(data_remainder, 1)

# Add control data

In [ ]:
# ------------------------------------ Choose Run parameters here -----------------------------------------------
OVERWRITE = False
RESULTS_PATH = os.path.join(dirname, "..", "data", "results.csv")
models = [
    ('RF', RandomForestClassifier, RF_hyperparams_list), 
    ('LR', LogisticRegression, LR_hyperparams_list)
]
tasks = ['los_3', 'los_7']
number_of_runs = 10
# ---------------------------------------------------------------------------------------------------------------

# Set up initial results dataframe
results = load_results(RESULTS_PATH)
progress = []
if OVERWRITE:
    prev_results_indices = results.loc[(results['Task'].isin(tasks)) &
                                       (results['Model'].isin([m for m,_,_ in models])) & 
                                       (results['Epsilon'] == "Control")].index
    results.drop(prev_results_indices, inplace=True) 
    
# Feature engineering pipeline 
data_remainder, Ys_remainder, data_test, Ys_test = create_test_sets(squashed_data, Ys)
data_train, data_dev, Ys_train, Ys_dev = create_train_dev_sets(data_remainder, Ys_remainder)
normalize_datasets(data_train, data_dev, data_test)

# These steps not necessary with squashing
# data_train, data_dev, data_test = [simple_imputer(df) for df in (data_train, data_dev, data_test)]
# data_flat_train, data_flat_dev, data_flat_test = flatten_arrays(data_train, data_dev, data_test)
data_flat_train, data_flat_dev, data_flat_test = data_train, data_dev, data_test

for df in data_train, data_dev, data_test: 
    assert not df.isnull().any().any()

# Use only mean columns
data_flat_train = data_flat_train.loc[:, pd.IndexSlice['mean', :]]
data_flat_dev = data_flat_dev.loc[:, pd.IndexSlice['mean', :]]
data_flat_test = data_flat_test.loc[:, pd.IndexSlice['mean', :]]  

# Run gridsearch
for _ in range(number_of_runs):
    for model_name, model, hyperparams_list in models:
            for task in tasks:
                SEED += 1
                np.random.seed(SEED)
                update_progress(progress, "Running model %s on target %s" % (model_name, task))
                result = run_basic(model, hyperparams_list, data_flat_train, data_flat_dev, data_flat_test, task, Ys_train, Ys_dev, Ys_test, progress)
                results = save_result(task, model_name, "Control", result, results, RESULTS_PATH)

                update_progress(progress, '\033[1m' + "Final results for model %s on target %s" % (model_name, task) + '\033[0m', replace=2)
                update_progress(progress, result.pop("Params"))
                update_progress(progress, result)
                update_progress(progress, "")


### Code for figures

In [ ]:
fe = fm.FontEntry(
    fname='Montserrat-Regular.ttf',
    name='Montserrat')
fm.fontManager.ttflist.insert(0, fe) # or append is fine
matplotlib.rcParams['font.family'] = fe.name # = 'your custom ttf font name'

In [ ]:
params = {"ytick.color" : "w",
          "xtick.color" : "w",
          "axes.labelcolor" : "w",
          "axes.edgecolor" : "w",
         "font.size": "20"}
plt.rcParams.update(params)

In [ ]:
fig, ax = plt.subplots()

fig.set_facecolor("k") 

ax.plot(fpr, tpr, linewidth=2, color="red", label="ROC Curve")


ax.plot([0, 1], [0, 1], label="Random Classifier", c="limegreen", linestyle="dashed")

ax.set_ylabel("True Positive Rate")
ax.set_xlabel("False Positive Rate")

ax.grid(which="major", axis="x")
ax.grid(which="major", axis="y")

ax.fill_between(fpr, np.zeros(len(fpr)), tpr, color='r', alpha=.25, label="AUROC")


ax.legend(loc=4, framealpha=1, labelcolor="w", facecolor="black")

y_locator = ticker.MultipleLocator(0.1)
ax.yaxis.set_major_locator(y_locator)
ax.xaxis.set_major_locator(y_locator)

fig.suptitle("Reciever Operating Characteristic curve of \nRandom Forest predicting 3 day length-of-stay", fontsize=24, color="w")
fig.set_size_inches(14, 10)
plt.show()
fig.savefig("ROCCURVE_RF" + ".png", transparent=True)

### OLD RF/LR


### GRU-D OLD

In [ ]:
FPR